# 라이브러리 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# mecab 설치 (1분 30초 정도 걸림)
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
# huggingface 설치
!pip install transformers

In [ ]:
from gensim.summarization.summarizer import summarize

from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import os
import json

# Data load

## json 데이터 읽기

In [ ]:
file_name = './playsotre_review_crawling/코끼리_dict'

with open(file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)
    print("")

{'user_1': {'id': '김계임', 'date': '2022년 5월 5일', 'rating': '4', 'review': '지난 2년동안 코끼리를 잘 사용하고 있습니다. 그런데 오늘 새벽에 업데이트를 실시한 후엔 앱을 사용할 수 없습니다. 업데이트 팝업이 사라지지 않아서 다른 기능들을 전혀 사용할 수가 없습니다. 재설치도 해보고 폰을 껐다 켜보아도 같은 상황입니다. 해결책을 알려주세요.'}, 'user_2': {'id': 'Google 사용자', 'date': '2019년 9월 1일', 'rating': '5', 'review': '사흘째 사용 중입니다. 알람과 백그라운드 기능도 잘 작동하고 무엇보다 매일 새롭게 배달되는 명상을 따라하기만하면 자연스럽게 명상이 생활의 일부가 되도록 설계되어 있어 참 좋습니다. (추가) 유료 전환 후 건의 사항이 있어서 기존 작성리뷰 수정합니다. 1. 알람으로 명상을 바로 접속해서 사용을 마쳤는데 계속 휴대폰 상단 메뉴에 아이콘이 떠 있습니다. 자동 숨김이나 마침 기능이 필요한 것 같고, 오류라면 수정해주셔야 할 것 같습니다. 2. 이어듣기 기능 추가 요청: 명상 클래스 목록에서 강의 하나를 들으면 종료 후 자동으로 다음 강의가 재생되는 기능이 있으면 좋겠습니다.'}, 'user_3': {'id': '도은구', 'date': '2020년 7월 16일', 'rating': '5', 'review': '(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다. 최근에 친한친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요. 감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만

In [ ]:
text = json_data['user_3']['review']
text

'(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다. 최근에 친한친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요. 감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만 끊김현상이 일어나서 별점한개를 뺐네요ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요 ㅎㅎ 그리고 데이터나 와이파이 환경에서 명상을 다운로드 받으면, 데이터나 와이파이가 없더라도 다운받은 목록들은 들을 수 있었으면 좋겠어요! 제 테블릿이 데이터가 없는 제품이라 운동할 때 숲에서 걷기명상을 듣고 싶은데 못들어서 조금 아쉽습니다ㅜㅜ 개선 가능한지 답변부탁드립니다!'

# 전처리
1. 주어 - 서술어 쌍으로 자르기!  
2. 의미없는 표현 자르기 
3.  POS가 5개 이하면 자르기


In [ ]:
okt = Okt()

def CustomTokenizer(corpus, stop = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']):
    tokenized = []
    for i, j in okt.pos(corpus, stem = True, norm = True):
        if j in stop:
            continue
        tokenized.append(i)
    return tokenized  

## 전처리 함수 설정

In [ ]:
#꼬꼬마분석기
kkma = Kkma()

#text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
def text2sentences(text):
    sentences = kkma.sentences(text)  #text일 때 문장별로 리스트 만듦
    for idx in range(0, len(sentences)):  #길이에 따라 문장 합침(위와 동일)
        
        # 문장의 품사가 5개 미만이면 생략
        if len(CustomTokenizer(sentences[idx])) <= 5:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''
  
    for idx in range(0, len(sentences)):
      
      word_list = []

      # ['Punctuation', 'Foreign'] 제거
      for word, pos in okt.pos(sentences[idx], norm = True):
        stop = ['Punctuation', 'Foreign']
        if pos in stop:
          word = ""
        # 문장을 예쁘게 하기 위해 임시로 ';' 붙여두기
        elif pos == 'Suffix' or pos == 'Josa':
          word = word + ';'
        word_list.append(word)

      # 문장 클리닝 
      sentence = ''
      for word in word_list:
        if word == '':
          continue
        elif ';' in word:
          sentence += word[:-1]
        else: 
          sentence += " " + word 

      sentences[idx] = sentence

    # 공란 문장 제거
    sentences__ = []
    for sentence in sentences:
      if sentence == '':
        pass
      else:
        sentences__.append(sentence)

    return sentences__

### user_3 리뷰로 확인

In [ ]:
sentences = text2sentences(json_data['user_3']['review'])
sentences

[' 코끼리 앱 주변에도 막 엄청 추천하고 다녀요',
 ' ㅎㅎ 명상 수면 심리 등등 도움이 엄청 많이 됩니다',
 ' 최근에 친한 친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요 감사합니다',
 ' 잘 때마다 명상 하면서 중간에 꿀 잠 잡니 다 짱이에요',
 ' ㅎㅎ 공부 할 때도 집중력 명상 들으면서 평소 하던 거에 1.5 배 더 공부 할 수 있었어요',
 ' ㅎ 다만 끊김 현상이 일어나서 별 점한 개를 뺐네요',
 ' ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요',
 ' ㅎㅎ 그리고 데이터나 와이 파이 환경에서 명상을 다운로드 받으면 데이터나 와이 파이가 없더라도 다운 받은 목록들은 들을 수 있었으면 좋겠어요',
 ' 제 테블릿이 데이터가 없는 제품이라 운동 할 때 숲에서 걷기 명상을 듣고 싶은데 못 들어서 조금 아쉽습니다',
 ' ㅜㅜ 개선 가능한지 답변 부탁드립니다 수정 넘 좋아요']

### 기존과 비교

In [ ]:
def text2sentences2(text):
    sentences = kkma.sentences(text)  #text일 때 문장별로 리스트 만듦
    for idx in range(0, len(sentences)):  #길이에 따라 문장 합침(위와 동일)
        
        # 문장의 품사가 5개 미만이면 생략
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''
        return sentences

In [ ]:
text2sentences2(json_data['user_3']['review'])

['( 수정) 넘 좋아요.....',
 '♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요',
 'ㅎㅎ 명상, 수면, 심리 등등 도움이 엄청 많이 됩니다.',
 '최근에 친한 친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요.',
 '감사합니다.',
 '♡ -- -- -- -- -- -- -- -- --‐ -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --- 잘 때마다 명상하면서 중간에 꿀 잠 잡니',
 '다 짱이에요',
 'ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던 거에 1.5 배 더 공부 할 수 있었어요',
 'ㅎ 다만 끊김 현상이 일어나서 별 점한 개를 뺐네요',
 'ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요',
 'ㅎㅎ 그리고 데이터나 와이 파이 환경에서 명상을 다운로드 받으면, 데이터나 와이 파이가 없더라도 다운 받은 목록들은 들을 수 있었으면 좋겠어요!',
 '제 테블릿이 데이터가 없는 제품이라 운동할 때 숲에서 걷기 명상을 듣고 싶은데 못 들어서 조금 아쉽습니다',
 'ㅜㅜ 개선 가능한지 답변 부탁드립니다!']

## 전체 리뷰 전처리 적용

In [ ]:
len(list(json_data.keys()))

1274

In [ ]:
for user in tqdm(list(json_data.keys())):
  json_data[user]['review'] = text2sentences(json_data[user]['review'])

json_data

  0%|          | 0/1274 [00:00<?, ?it/s]

{'user_1': {'id': '김계임',
  'date': '2022년 5월 5일',
  'rating': '4',
  'review': [' 지난 2년 동안 코끼리를 잘 사용하고 있습니다',
   ' 그런 데 오늘 새벽에 업데이트를 실시한 후엔 앱을 사용 할 수 없습니다',
   ' 업데이트 팝업이 사라지지 않아서 다른 기능들을 전혀 사용 할 수가 없습니다',
   ' 재 설치도 해보고 폰을 껐다 켜 보아도 같은 상황 입니다 해결 책을 알려 주세요']},
 'user_2': {'id': 'Google 사용자',
  'date': '2019년 9월 1일',
  'rating': '5',
  'review': [' 알람과 백 그 라운드 기능도 잘 작동하고 무엇보다 매일 새롭게 배달 되는 명상을 따라하기만 하면 자연스럽게 명상이 생활의 일부가 되도록 설계 되어 있어 참 좋습니다',
   ' 추가 유료 전환 후 건의 사항이 있어서 기존 작성 리뷰 수정 합니다',
   ' 1 알람으로 명상을 바로 접속 해서 사용을 마쳤는데 계속 휴대폰 상단 메뉴에 아이콘이 떠 있습니다',
   ' 자동 숨 김이나 마침 기능이 필요한 것 같고 오류 라면 수정 해 주셔야 할 것 같습니다',
   ' 2 이어 듣기 기능 추가 요청 명상 클래스 목록에서 강의 하나를 들으면 종료 후 자동으로 다음 강의가 재생 되는 기능이 있으면 좋겠습니다 사흘째 사용 중 입니다']},
 'user_3': {'id': '도은구',
  'date': '2020년 7월 16일',
  'rating': '5',
  'review': [' 코끼리 앱 주변에도 막 엄청 추천하고 다녀요',
   ' ㅎㅎ 명상 수면 심리 등등 도움이 엄청 많이 됩니다',
   ' 최근에 친한 친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요 감사합니다',
   ' 잘 때마다 명상 하면서 중간에 꿀 잠 잡니 다 짱이에요',
   ' ㅎㅎ 공부 할 때도 집중력 명상 들으면서 평소 하던 거에 1.5

In [ ]:
json_df = pd.DataFrame(json_data['user_1'].values()).T
json_df.columns = list(json_data['user_1'].keys()) # 칼럼명 지정 

In [ ]:
json_df

,id,date,rating,review
0,김계임,2022년 5월 5일,4,"[ 지난 2년 동안 코끼리를 잘 사용하고 있습니다, 그런 데 오늘 새벽에 업데이트..."


## 리뷰 묶음 처리

In [ ]:
file_name = './리뷰 묶음 추출'

with open(file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)
    print("")

In [ ]:
len(json_data)

6000

In [ ]:
sentences__ = [] 

for i in tqdm(range(len(json_data))):
  changed_sentences = text2sentences(json_data[i])

  for sentence in changed_sentences:
    if len(sentence) == 0:
      continue
    else: 
      sentences__.append(sentence)
sentences__

  0%|          | 0/6000 [00:00<?, ?it/s]

[' 사기 앱 탈퇴 못 하게 되어 잇어요',
 ' 이 거 완전 신종 사기 자동 결제 뜨고 머지 회원 탈퇴 하는 거 찾으러 아무리 찾아봐도 없고 만약 결제 되면 신고 할 겁니다',
 ' 유료로 전환 되는 거 모르고 설치 했습니다',
 ' 기간 이후 결제 되지 않도록 탈퇴 시켜 주세요 로그 아웃밖에 안되는데 확인 부탁드립니다',
 ' 광고 보고 왔는데 광고에 나온 부분은 어디에 있는 거죠 도통 보이질 않는군요',
 ' 그리고 유료가 아니면 볼 수 있는 게 별로 없네요',
 ' 좋은데 애용 하는데 자꾸 끊기고 꺼져 요 잠들 쯤 그러면 너무 짜증나요',
 ' 이 개 XX 들아 7일 후 자동 결제면 사용 할지 말지 선택권이나 줘라 버튼이 계속밖에 없냐 욕 나오게 만드네',
 ' 계속 버튼 안 누르고 바로 뒤로 가기 눌렀는데 결제 되기만 해 봐라 가만 안 둔다',
 ' 일주일 뒤 자동 결제네요 사용 안 했으니 탈퇴 해 주세요 뭐 이런 앱이 있나요 짜증나게',
 ' 구독 취소 했습니다',
 ' 환불 해 주세요 결제 취소 해 주세요',
 ' 머 이런 장사꾼 앱이 구독 취소 해 주시고 회원 탈퇴 시켜 주세요',
 ' 앱을 어떻게 다운 받아서 설치 하는 플레이 스토어에서 설치 했는데 여는 방법을 모르겠어요 ㅠㅠ 내가 기계 치 맞나',
 ' 봐요',
 ' 어찌 열고 하는지 말씀 좀 주세요',
 ' 아니 근데 짜증나는 건 왜 7일 무료 라면서 결제 하래요',
 ' 저 텐트에서 비오는 거 듣고 싶었는데 불쾨 하네요',
 ' 아 쓰래 기 앱 임 돈내라 하고 하나도 없어 절대 설치 마 삼',
 ' 서 깔았는데 필요가 없어서 한참 안 쓰다가 생각 나서 지울 랬 는 데 자동으로 결제 되고 있었다네요',
 ' ㅋㅋㅋ 결제 취소 방법을 모르겠어서 그런데 결제 취소 해 주시고 제 정보 아예 지워 주셨으면 좋겠습니다 김 리뷰가 좋대',
 ' 아니 공짜처럼 해 두구 3일 뒤 50000원 결제 뜨길래 아무 버튼 안 누르고 바로 나가서 지웠는데 결제 되 진 않겠죠',
 ' 결제 안된 걸 알려면 어뜩 게

In [ ]:
sentences_df = pd.DataFrame(sentences__)
sentences_df.columns = ['Review']

sentences_df

,Review
0,사기 앱 탈퇴 못 하게 되어 잇어요
1,이 거 완전 신종 사기 자동 결제 뜨고 머지 회원 탈퇴 하는 거 찾으러 아무리 찾...
2,유료로 전환 되는 거 모르고 설치 했습니다
3,기간 이후 결제 되지 않도록 탈퇴 시켜 주세요 로그 아웃밖에 안되는데 확인 부탁드립니다
4,광고 보고 왔는데 광고에 나온 부분은 어디에 있는 거죠 도통 보이질 않는군요
...,...
7200,업 뎃 하기 전에 구매 해 놀 껄 돈 받는 시스템이 바 껴서 더 비싸지 무 ㅜㅜㅜ
7201,요즘 불면증인지 잠이 안 오는데 이 거 쓰고 굿 입니다 아 삭 하 누
7202,HD라 그런지 해상도 두 좋구 괜찮습니다
7203,평온한 마음이 듭니다


In [ ]:
path = './data/'
title = 'Labeling해보자.xlsx'

sentences_df.to_excel(excel_writer= path + title)

# Text_Rank 적용

In [ ]:
from gensim.summarization.summarizer import summarize

In [ ]:
import os
import json

file_name = './playsotre_review_crawling/코끼리_dict'

with open(file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)
    print("")

{'user_1': {'id': '김계임', 'date': '2022년 5월 5일', 'rating': '4', 'review': '지난 2년동안 코끼리를 잘 사용하고 있습니다. 그런데 오늘 새벽에 업데이트를 실시한 후엔 앱을 사용할 수 없습니다. 업데이트 팝업이 사라지지 않아서 다른 기능들을 전혀 사용할 수가 없습니다. 재설치도 해보고 폰을 껐다 켜보아도 같은 상황입니다. 해결책을 알려주세요.'}, 'user_2': {'id': 'Google 사용자', 'date': '2019년 9월 1일', 'rating': '5', 'review': '사흘째 사용 중입니다. 알람과 백그라운드 기능도 잘 작동하고 무엇보다 매일 새롭게 배달되는 명상을 따라하기만하면 자연스럽게 명상이 생활의 일부가 되도록 설계되어 있어 참 좋습니다. (추가) 유료 전환 후 건의 사항이 있어서 기존 작성리뷰 수정합니다. 1. 알람으로 명상을 바로 접속해서 사용을 마쳤는데 계속 휴대폰 상단 메뉴에 아이콘이 떠 있습니다. 자동 숨김이나 마침 기능이 필요한 것 같고, 오류라면 수정해주셔야 할 것 같습니다. 2. 이어듣기 기능 추가 요청: 명상 클래스 목록에서 강의 하나를 들으면 종료 후 자동으로 다음 강의가 재생되는 기능이 있으면 좋겠습니다.'}, 'user_3': {'id': '도은구', 'date': '2020년 7월 16일', 'rating': '5', 'review': '(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다. 최근에 친한친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요. 감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만

In [ ]:
text = json_data['user_3']['review']
text

'(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다. 최근에 친한친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요. 감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만 끊김현상이 일어나서 별점한개를 뺐네요ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요 ㅎㅎ 그리고 데이터나 와이파이 환경에서 명상을 다운로드 받으면, 데이터나 와이파이가 없더라도 다운받은 목록들은 들을 수 있었으면 좋겠어요! 제 테블릿이 데이터가 없는 제품이라 운동할 때 숲에서 걷기명상을 듣고 싶은데 못들어서 조금 아쉽습니다ㅜㅜ 개선 가능한지 답변부탁드립니다!'

In [ ]:
print(summarize(text, ratio=0.3))

감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만 끊김현상이 일어나서 별점한개를 뺐네요ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요 ㅎㅎ 그리고 데이터나 와이파이 환경에서 명상을 다운로드 받으면, 데이터나 와이파이가 없더라도 다운받은 목록들은 들을 수 있었으면 좋겠어요!


In [ ]:
print(summarize(text, ratio=0.5))

(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다.
최근에 친한친구가 사고로 하늘나라로 갔는데 그때 상실의 아픔에 관한 명상이 굉장히 도움이 많이 됐어요.


In [ ]:
text = ''
for i in range(10):
  i +=  1 
  text += json_data['user_' + str(i)]['review']

In [ ]:
print(summarize(text))

2. 이어듣기 기능 추가 요청: 명상 클래스 목록에서 강의 하나를 들으면 종료 후 자동으로 다음 강의가 재생되는 기능이 있으면 좋겠습니다.(수정) 넘 좋아요.....♡ 코끼리 앱 주변에도 막 엄청 추천하고 다녀요 ㅎㅎ 명상, 수면, 심리등등 도움이 엄청 많이 됩니다.
감사합니다.♡------------------‐-------------------------------------------------------------잘 때마다 명상하면서 중간에 꿀잠 잡니다 짱이에요 ㅎㅎ 공부할 때도 집중력명상들으면서 평소하던거에 1.5배 더 공부 할 수 있었어요 ㅎ 다만 끊김현상이 일어나서 별점한개를 뺐네요ㅜㅜ 끊기는 것만 해결 됐으면 좋겠어요 ㅎㅎ 그리고 데이터나 와이파이 환경에서 명상을 다운로드 받으면, 데이터나 와이파이가 없더라도 다운받은 목록들은 들을 수 있었으면 좋겠어요!
오래 오래 계속 잘 있어주면 좋겠어요 데이터 부담이 없도록 오프라인에서 들을 수 있도록 휴대폰에 저장할 수 있는 기능이 있으면 좋겠어요.
복잡하고 불안정한 마음이 상당히 안정되고 편안해지고 있어요 ^^자기 전에 항상 잘 사용하고 있습니다.
그런데 홈화면에 내가 좋아요한 목록이나 화면 하단에 좋아요 모음 버튼이 생겼으면 좋겠습니다.
이번 업데이트도 너무 좋아요 무료로 들을수있는 컨텐츠들이 더 생기고 깔끔해졌어요 😊 앞으로도 애용할게요 감사합니다!속도 제한 데이터 요금제를 사용하고 있는 고등학생입니다.


# BerTOPIC 적용

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
topic_model = BERTopic(language="korean", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(sentences__)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/226 [00:00<?, ?it/s]

2022-09-21 13:40:21,410 - BERTopic - Transformed documents to Embeddings
2022-09-21 13:41:06,624 - BERTopic - Reduced dimensionality
2022-09-21 13:41:09,510 - BERTopic - Clustered reduced embeddings


In [ ]:
print(topics)

[17, -1, -1, -1, 41, 70, 0, 8, -1, -1, 77, -1, 22, -1, 57, -1, 8, -1, 1, -1, -1, 11, -1, 14, 1, 1, -1, 1, 17, -1, 27, -1, 20, 4, 0, 40, 5, -1, 35, 28, 1, 80, 14, -1, -1, -1, 14, -1, 24, -1, 11, -1, 19, -1, -1, -1, 1, -1, 1, 34, -1, 34, -1, 5, 23, 0, -1, 27, -1, -1, 4, 50, 22, -1, -1, -1, 13, 29, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 67, -1, 11, 21, 5, -1, 54, -1, 63, -1, -1, -1, 13, -1, -1, 62, -1, -1, -1, -1, 5, 19, 11, -1, 13, 5, -1, -1, -1, -1, 5, 11, 67, 60, -1, 17, -1, -1, -1, -1, 2, -1, -1, 8, 25, 23, 1, -1, 11, 54, -1, 1, -1, 50, -1, 84, -1, -1, 8, -1, 74, 0, 25, 40, -1, -1, 24, 6, -1, -1, 11, -1, 5, -1, 14, 1, 2, 78, -1, 5, -1, -1, 1, -1, -1, 6, 25, -1, 17, 19, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 5, -1, -1, 14, -1, -1, -1, -1, -1, 8, -1, 1, -1, 60, -1, -1, 8, -1, -1, 8, 68, 1, 13, -1, 56, -1, 15, 11, 21, 8, -1, 76, -1, 60, -1, 1, 29, 1, 1, 29, -1, 8, -1, -1, 8, 21, -1, 20, 20, 4, -1, 11, 5, -1, 22, 15, -1, 14, -1, -1, 35, -1, 2, -1, -1, -1, 80, 1, -1, -1, 5,

In [ ]:
print(probs)

[[4.60820371e-003 6.73108022e-003 5.65657109e-003 ... 7.12446560e-003
  9.73184994e-003 7.82115750e-003]
 [3.15244072e-003 1.12731763e-002 5.56799038e-003 ... 8.78438691e-003
  1.91419667e-002 9.66462512e-003]
 [2.90662795e-003 1.50453833e-002 1.37294528e-002 ... 3.25677844e-002
  5.91652494e-003 2.61313088e-003]
 ...
 [8.38965749e-003 9.43920323e-003 1.54033593e-002 ... 1.00010462e-002
  6.27023633e-003 3.96578456e-003]
 [2.45810593e-307 1.58196778e-307 2.23335559e-307 ... 1.67639486e-307
  1.11866412e-307 7.78785839e-308]
 [1.02655956e-002 9.43140756e-003 1.44030333e-002 ... 9.76966134e-003
  6.14406714e-003 3.95642647e-003]]


In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name
0,-1,3126,-1_결제_취소_자동_주세요
1,0,694,0_잠이_잠을_수면_자고
2,1,291,1_ㅠㅠ_취소_주세요_안되게
3,2,153,2_ㅋㅋㅋ_키보고_ㅋㅋ_맘에
4,3,109,3_집중_마음이_호흡_집중력
...,...,...,...
84,83,10,83_확인_forest_부탁_수령을
85,84,10,84_인전에_절이라도_실행하고_놈들
86,85,10,85_어디다_뜨던데_떨어져_지우게
87,86,10,86_무료_체험_싶어서요_서요


In [ ]:
freq['Name'][0]

'-1_결제_취소_자동_주세요'

In [ ]:
for i in range(freq.shape[0]):
  print(freq['Name'][i])

-1_결제_취소_자동_주세요
0_잠이_잠을_수면_자고
1_ㅠㅠ_취소_주세요_안되게
2_ㅋㅋㅋ_키보고_ㅋㅋ_맘에
3_집중_마음이_호흡_집중력
4_시간_시간을_설정_알람
5_환불_요청_결제가_했는데
6_want_to_please_my
7_좋은_주셔서_앱인_만들어
8_7일_자동_된다고_결제
9_감사합니다_감사_주셔서_좋은
10_좋아요_쓰고_너무_있어요
11_결제가_결제_8월_무료
12_좋아요_진짜_좋은_아주
13_삭제_계정_로그_탈퇴
14_7일_무료_자동_체험
15_종료_app_it_the
16_소리_소리도_녹음_좋겠어요
17_쓰레기_앱이_앱이네_이런
18_앱이_앱은_this_app
19_환불_주세요_당장_해지하고
20_없나요_불편합니다_최종_별도
21_탈퇴_없네요_없습니다_되요
22_계정_삭제_주시고_안되게
23_됐네요_결제가_결제_거죠
24_녹음_녹음이_다른_소리가
25_드립니다_부탁_빨리_해결
26_다양한_소리_백색_소리를
27_아이_폰을_핸드폰_전화
28_개선_기능이_컨텐츠도_반영
29_계정_삭제_탈퇴_가입
30_너무하네요_실망_좋은지_그게
31_음악_음악이_음악도_악기
32_아이_핸드폰을_바꾸고_폰으로
33_코끼리_코끼리를_편안함과_불면증과
34_되었네요_결제_결제가_자여
35_구독_결제_자동_다운하
36_환불_부탁드립니다_주세요_결제라
37_좋아요_아주_사랑_잠재우기에
38_건물이_무너지지_집이_마을
39_가나_스님의_도움이_많은
40_무료_버전_무료로_유료
41_광고_광고가_광고로_보고
42_매일_좋습니다_명상_감사해요
43_ㅋㅋ_ㅎㅎ_ㅠㅠ_ㅎㅎㅎ
44_소리가_중간에_시끄럽게_거슬리는
45_달에_굳이_1년_가격이
46_구매_없는데_몰랐네요_겁니까
47_감사합니다_하셨어요_수고_제작자님
48_피아노_조절_곡이_소리랑
49_35000원_나갔네요_빠져나갔어요_35000
50_리뷰들_리뷰_리뷰에_보고
51_좋아요_안녕하세요_건강하세요_환급
52_인이_로그_권한이_접근
53_마인드_엠씨_다양한_소리
54_드립니다_부탁_있다네요_습니다
55_the_소리를_to_있

## pos 분리 이전 기준으로 BerTopic

In [ ]:
sentences__2 = [] 

for i in tqdm(range(len(json_data))):
  changed_sentences = text2sentences2(json_data[i])

  for sentence in changed_sentences:
    if len(sentence) == 0:
      continue
    else: 
      sentences__2.append(sentence)
sentences__2

  0%|          | 0/6000 [00:00<?, ?it/s]

['사기 앱 탈퇴 못하게 되어 잇어요...',
 '이거 완전 신종 사기 자동 결제 뜨고 머지???? 회원 탈퇴 하는 거 찾으러 아무리 찾아봐도 없고 만약 결제되면 신고 할 겁니다.',
 '유료로 전환되는 거 모르고 설치했습니다.',
 '기간이후 결제되지 않도록 탈퇴시켜 주세요.',
 '로그 아웃밖에 안되는데 확인 부탁드립니다.',
 '광고 보고 왔는데 광고에 나온 부분은 어디에 있는 거죠?',
 '도통 보이질 않는군요',
 '그리고 유료가 아니면 볼 수 있는 게 별로 없네요',
 '좋은데.. 애용하는데.. 자꾸 끊기고 꺼져 요 잠들 쯤 그러면 너무 짜증나요...',
 '오히려 잠이 않 오게 하네요',
 '이 개 XX 들아 7일 후 자동 결제면 사용할지 말지 선택권이나 줘 라 버튼이 계속밖에 없냐',
 '욕 나오게 만드네',
 '계속 버튼 안 누르고 바로 뒤로 가기 눌렀는데 결제되기만 해 봐라 가만 안 둔다',
 '일주일 뒤 자동 결제네요 사용 안했으니 탈퇴해 주세요 뭐이런앱이있나요 짜증나게',
 '결제 취소해 주세요.',
 '구독 취소했습니다.',
 '환불해 주세요.',
 '머 이런 장사꾼앱이 구독 취소 해 주시고 회원 탈퇴시켜 주세요',
 '앱을 어떻게 다운 받아서 설치하는 플레이 스토어에서 설치했는데 여는 방법을 모르겠어요',
 'ㅠㅠ 내가 기계치 맞나',
 '봐요',
 '어찌 열고 하는지 말씀 좀 주세요',
 '아니 근데 짜증나는 건 왜 7일 무료 라면서 결제 하래요?',
 '저 텐트에서 비오는 거 듣고 싶었는데 불쾨하네요',
 '아 쓰래기 앱 임 돈내 라 하고 하나도 없어. 절대 설치 마 삼.',
 '서 깔았는데 필요가 없어서 한참 안 쓰다가 생각 나서 지울 랬 는 데 자동으로 결제되고 있었다네요',
 'ㅋㅋㅋㅋ 결제 취소 방법을 모르겠어서 그런데 결제 취소해 주시고 제 정보 아예 지워 주셨으면 좋겠습니다. 김 리뷰가 좋대',
 '소리가 안 들립니다.',
 '아니 공짜처럼 해 두구 3일 뒤 50000원 결제 뜨길래 아무 버튼 안 누르고 바로 나가서 지웠

## 시각화

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.005)

In [ ]:
topic_model2 = BERTopic(language="korean", calculate_probabilities=True, verbose=True)
topics2, probs2 = topic_model2.fit_transform(sentences__2)

Batches:   0%|          | 0/304 [00:00<?, ?it/s]

2022-09-21 13:52:42,036 - BERTopic - Transformed documents to Embeddings
2022-09-21 13:53:10,269 - BERTopic - Reduced dimensionality
2022-09-21 13:53:24,445 - BERTopic - Clustered reduced embeddings


In [ ]:
freq2 = topic_model2.get_topic_info()
freq2

,Topic,Count,Name
0,-1,4005,-1_자동_결제_구독_주세요
1,0,248,0_잠이_잠잘_자고_와요
2,1,173,1_환불_refund_환불은_환불해
3,2,140,2_결제가_되었어요_결제_정기
4,3,117,3_있어요_쓰고_좋네요_좋아요
...,...,...,...
164,163,10,163_음악이_음악_부담스럽게_다가오네요
165,164,10,164_실행이_않다_오류도_일인지
166,165,10,165_계정_회신이_계정정보를_눌렀었지만
167,166,10,166_회원_가입도_돼는데_구독하려는


In [ ]:
for i in range(freq2.shape[0]):
  print(freq2['Name'][i])

-1_자동_결제_구독_주세요
0_잠이_잠잘_자고_와요
1_환불_refund_환불은_환불해
2_결제가_되었어요_결제_정기
3_있어요_쓰고_좋네요_좋아요
4_완전_같아용_악기_드려요
5_잠을_잠이_잠드는_자는데
6_모르겠어요_되요_권한이_유효하지
7_황당하네요_불편하네요_불편합니다_너무
8_소리_다양한_소리를_백색
9_아이_드로_폰을_핸드폰을
10_구독_취소해_취소를_취소
11_녹음이_녹음_소리가_소리
12_힐링_비소리_감사하겠습니다_ㅎㅎㅎ
13_환불해_환불_주세요_당장
14_재생이_성공_현상이_플레이가
15_볼게요_도움_나를_앞으로
16_같습니다_좋겠네요_있었으면_기능이
17____
18_시간_시간을_알람_7시
19_계정_삭제_delete_삭제해
20_good_아주_좋습니다_괜찮은
21_7일_자동_라니_이후
22_좋은_앱은_앱이에요_앱이
23_잠이_와요_솔솔_분만에
24_앱을_자고_잠에_불면증
25_빗소리_빗소리를_빗소리가_빗소리도
26_스님_혜민_정은님_지도
27_무료체험_7일_무료체험도_무료
28_불편합니다_불편해요_나네요_힘들어요
29_음악도_음악_음악을_음악이
30_마세요_사용하지_없습니다_않습니다
31_감사합니다_고맙습니다_항상_감사
32_전화_아이_핸드폰_폰을
33_탈퇴해_결제하지_없고_주세요
34_건물이_건물_건물을_구경할
35_호수_웃긴_내기하고_장난칩니까
36_취소해_결제_취소는_요청합니다
37_application_won_to_want
38_한글_ㅎㅎ_줘야_멤버들
39_수면_수면의_깊은_수면질
40_무료로_무료로도_무료_유료
41_도움이_됩니다_꾸준히_도움
42_사용하고_유용하게_있습니다_useful
43_ㅠㅠ_하트_안함_사용
44_입니다_5개_번째네요_s10
45_무료체험_결제가_되었습니다_찾다
46_보냈습니다_메일을_메일_메일도
47_payment_cancel_account_my
48_마음이_가나_편해_마음을
49_감사합니다_감사해요_고맙습니다_마스
50_환불_요청합니다_요구합니다_신청합니다
51_ㅋㅋ_ㅋㅋㅋ_ㅋㅋㅋㅋ_좋아요
52_어떻게_해

In [ ]:
topic_model2.visualize_distribution(probs[200], min_probability=0.005)

# KeyBERT 적용

In [ ]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.8 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 


In [ ]:
from keybert import KeyBERT

keywords_dict = {}
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')


for i in tqdm(range(len(sentences__[:3]))):
  keywords = kw_extractor.extract_keywords(sentences__[i])
  keywords_dict['review_'+ str(i)] = keywords

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
keywords_dict

{'review_0': [('사기', 0.8784),
  ('하게', 0.8726),
  ('탈퇴', 0.4905),
  ('잇어요', 0.4905),
  ('되어', 0.4905)],
 'review_1': [('겁니다', 0.7696),
  ('신종', 0.7323),
  ('완전', 0.7147),
  ('자동', 0.7141),
  ('결제', 0.6987)],
 'review_2': [('모르고', 0.8078),
  ('유료로', 0.8001),
  ('전환', 0.7956),
  ('설치', 0.7077),
  ('했습니다', 0.2427)]}

In [ ]:
from keybert import KeyBERT

keywords_dict2 = {}
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')


for i in tqdm(range(len(sentences__2[:3]))):
  keywords = kw_extractor.extract_keywords(sentences__2[i])
  keywords_dict2['review_'+ str(i)] = keywords

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
keywords_dict2

{'review_0': [('사기', 0.9089),
  ('탈퇴', 0.6807),
  ('잇어요', 0.6807),
  ('못하게', 0.6807),
  ('되어', 0.6807)],
 'review_1': [('겁니다', 0.7987),
  ('신종', 0.7658),
  ('자동', 0.7497),
  ('완전', 0.7486),
  ('결제', 0.7284)],
 'review_2': [('모르고', 0.9085),
  ('유료로', 0.9001),
  ('전환되는', 0.4579),
  ('설치했습니다', 0.4579)]}